### **6.2 - Minimising Repetitive patterns with SubDAGs**

Cuando empiezas a construir DAGs más complejos puedes acabar con muchas tareas para ejecutar en paralelo que tienen la misma función lógica.  Por ejemplo, puedes tener tres tareas descargando datos de diferentes fuentes.  Y una cosa que puedes querer es hacer tu DAG más limpio agrupando esas tareas.  La forma de resolver esto es utilizando SubDAGs.  Permíteme empezar por el siguiente caso de uso.  

<center><img src="https://i.postimg.cc/vBL3dH2n/a924.png"></center>

Supongamos que tienes un DAG como este, con múltiples tareas que pueden ejecutarse en paralelo.  En este ejemplo, todas las tareas con el prefijo s-1 o s-2, podrían ser visualmente agrupadas para hacer este DAG más limpio. Usando SubDAGs, terminaremos con el siguiente DAG donde las tareas s-1 y s-2 están visualmente combinadas en su propia caja. 

<center><img src="https://i.postimg.cc/bryLYc4B/a925.png"></center>

Como veremos más adelante en la interfaz de usuario, un SubDAG puede identificarse por los bordes en negrita que rodean la caja que contiene las tareas.  Eso es lo que puede ver aquí y aquí.  Tenga en cuenta que nada ha cambiado, las dependencias se mantienen, es más o menos sólo visual.  Para crear un SubDAG codificaremos un método de fábrica (factory method) que devuelva un objeto DAG con las tareas que queremos combinar.  A continuación, instanciaremos un SubDagOperator para adjuntar este SubDag al DAG principal.  Así es como Airflow diferencia entre un DAG y un SubDag.

<center><img src="https://i.postimg.cc/Y2Rbn0WK/a926.png"></center>

Antes de pasar a la práctica, tenga en cuenta estos puntos importantes. El SubDag sigue siendo gestionado por el DAG principal, por lo que debe mantener la misma start_date y el mismo scheduling_interval que su padre, de lo contrario puede terminar con comportamientos inesperados.  También el estado del SubDagOperator y las tareas son independientes.  Por ejemplo, un SubDagOperator marcado como exitoso (success) no afectará a las tareas subyacentes que aún pueden ser marcadas como fallidas (failed).  Por último, el mayor problema de los SubDags son los bloqueos (deadlocks).  De hecho, es posible especificar un executor en SubDags que se establece por defecto como Sequential Executor.  Si está utilizando Airflow en producción con SubDags y el Celery Executor está configurado, puede tener problemas.  De hecho, cuando un SubDagOperator se dispara, toma un worker slot y cada tarea en el DAG hijo toma un slot también hasta que todo el SubDAG está terminado.  Si recuerdas la sección 5, cada vez que se dispara una tarea, se toma un slot del pool por defecto (default pool) que está limitado a 128 por defecto.  Si no hay más slots disponibles, puede tener problemas.  Esto puede ralentizar el procesamiento de las tareas o incluso bloquear el DAG.  Una forma de evitar bloqueos es crear una cola sólo para SubDags.  Ok, sé que puedes estar perdido, pero no te preocupes te voy a mostrar un ejemplo en un minuto. Así que tómate un descanso rápido y nos vemos en el próximo vídeo.  

<center><img src="https://i.postimg.cc/Kj0fRp2v/a927.png"></center>